### 1) Clean and Analyse the Dataset

* Create a Jupyter Notebook `data_analysis.ipynb` where you clean the dataset and analyze the relation between Sale Price and quantity for some products.

* Save the cleaned dataset into `sales_data_cleaned.csv`.

In [9]:
import os
import pandas as pd
import logging

In [10]:
file_path = os.path.join("../data/raw/sales_data.csv")
cleaned_file_path = os.path.join("../data/processed/sales_data_cleaned.csv")
log_file_path = os.path.join("../log/sales_data_cleaning.log")

In [11]:
# Configure logging
logging.basicConfig(filename=log_file_path, level=logging.INFO)

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path, sep=";")

# log the number of rows and columns
logging.info(f"Initial number of rows: {df.shape[0]}")
logging.info(f"Initial number of columns: {df.shape[1]}")

# Rename columns
df.rename(
    columns={
        "TransactionID": "transaction_id",
        "ProductID": "product_id",
        "Quantity": "quantity",
        "SalePrice": "sale_price",
        "PurchasePrice": "purchase_price",
    },
    inplace=True,
)


logging.info("Columns renamed successfully.")


# Function to log dropped rows' transaction IDs
def log_dropped_rows(df, operation):
    dropped_transaction_ids = df.index.tolist()
    logging.info(
        f"{len(dropped_transaction_ids)} rows dropped during {operation}: {dropped_transaction_ids}"
    )


# Drop non-numeric rows from all columns
for col in df.columns:
    non_numeric_rows = df[pd.to_numeric(df[col], errors="coerce").isnull()]
    log_dropped_rows(non_numeric_rows, f"dropping non-numeric rows in column '{col}'")
    df = df[pd.to_numeric(df[col], errors="coerce").notnull()]


# Drop rows with missing values
missing_values = df[df.isnull().any(axis=1)]
df = df.dropna()
log_dropped_rows(missing_values, "dropping rows with missing values")

# Drop duplicate rows
duplicate_rows = df[df.duplicated()]
df = df.drop_duplicates()
log_dropped_rows(duplicate_rows, "dropping duplicate rows")

# Convert columns to the correct data types
df["transaction_id"] = df["transaction_id"].astype(int)
df["product_id"] = df["product_id"].astype(int)
df["quantity"] = df["quantity"].astype(int)
df["sale_price"] = df["sale_price"].astype(float)
df["purchase_price"] = df["purchase_price"].astype(float)


logging.info("Columns converted to correct data types.")

logging.info(f"Number of rows after cleaning: {df.shape[0]}")
logging.info(f"Number of columns after cleaning: {df.shape[1]}")

In [12]:
# Save the cleaned data to a new CSV file
df.to_csv(cleaned_file_path, index=False)

print("Data cleaning is done and saved in the file: ", cleaned_file_path)
print("Log file is saved in the file: ", log_file_path)

Data cleaning is done and saved in the file:  ../data/processed/sales_data_cleaned.csv
Log file is saved in the file:  ../log/sales_data_cleaning.log
